### Web Scraping


1-Shekh Ibn Baz offcial website 

In [ ]:
import requests
from bs4 import BeautifulSoup
import random
import time
import csv
import os

BASE_URL = "https://binbaz.org.sa"
START_PAGE = BASE_URL + "/fatwas/kind/1?page={}"
OUTPUT_FILE = "fatwas_binbaz.csv"

HEADERS_LIST = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.2 Safari/605.1.15",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:114.0) Gecko/20100101 Firefox/114.0",
]

# ---------------------- دوال مساعدة ----------------------

def fetch_page(url, retries=5):
    for attempt in range(retries):
        try:
            headers = {"User-Agent": random.choice(HEADERS_LIST)}
            response = requests.get(url, headers=headers, timeout=10)
            if response.status_code == 200:
                return response.text
            print(f"[ERROR] Status {response.status_code}, retry {attempt+1}/{retries}")
        except requests.exceptions.RequestException as e:
            print(f"[ERROR] Connection failed ({e}), retry {attempt+1}/{retries}")
        time.sleep(random.uniform(2, 5))
    return None

def normalize_url(url):
    if url.startswith("http"):
        return url
    return BASE_URL + url

def extract_answer(url):
    html = fetch_page(url)
    if not html:
        return ""
    soup = BeautifulSoup(html, "html.parser")
    body = soup.select_one(".fatwa__body")
    if body:
        return body.get_text("\n", strip=True)
    article = soup.find("article")
    if article:
        return article.get_text("\n", strip=True)
    return soup.get_text("\n", strip=True)

def parse_fatwas(html):
    soup = BeautifulSoup(html, "html.parser")
    articles = soup.select("article.fatwa")
    fatwas = []
    for art in articles:
        h1 = art.find("h1")
        if not h1:
            continue
        a = h1.find("a")
        if not a:
            continue
        question = a.get_text(strip=True)
        link = normalize_url(a["href"])
        answer = extract_answer(link)
        fatwas.append((question, answer, link))
    return fatwas

def save_to_csv(data, header=False):
    with open(OUTPUT_FILE, "a", encoding="utf-8-sig", newline="") as f:
        writer = csv.writer(f)
        if header and not os.path.exists(OUTPUT_FILE):
            writer.writerow(["question", "answer", "URL"])
        writer.writerows(data)

# ---------------------- Scraping كامل ----------------------

def main():
    print("Starting full-site scraping...\n")

    page = 1
    empty_pages = 0

    # إنشاء الملف مع Header لو مش موجود
    if not os.path.exists(OUTPUT_FILE):
        save_to_csv([], header=True)

    while True:
        url = START_PAGE.format(page)
        print(f"[INFO] Fetching page {page} ...")

        html = fetch_page(url)
        if not html:
            print(f"[ERROR] Failed to load page {page}. Skipping.")
            page += 1
            continue

        fatwas = parse_fatwas(html)
        if not fatwas:
            empty_pages += 1
            print(f"[INFO] No data found on page {page} (#{empty_pages}/3)")
            if empty_pages >= 3:
                print("No more fatwas found. Stopping.")
                break
        else:
            empty_pages = 0
            save_to_csv(fatwas)  # حفظ كل صفحة مباشرة
            print(f"[SUCCESS] Saved {len(fatwas)} fatwas from page {page}")

        time.sleep(random.uniform(1.5, 4))  # تأخير عشوائي
        page += 1

    print("\nScraping complete! All fatwas saved to:", OUTPUT_FILE)

# ---------------------- Run ----------------------

if __name__ == "__main__":
    main()


Starting full-site scraping...

[INFO] Fetching page 1 ...
[SUCCESS] Saved 15 fatwas from page 1
[INFO] Fetching page 2 ...
[SUCCESS] Saved 15 fatwas from page 2
[INFO] Fetching page 3 ...
[SUCCESS] Saved 15 fatwas from page 3
[INFO] Fetching page 4 ...
[SUCCESS] Saved 15 fatwas from page 4
[INFO] Fetching page 5 ...
[SUCCESS] Saved 15 fatwas from page 5
[INFO] Fetching page 6 ...
[SUCCESS] Saved 15 fatwas from page 6
[INFO] Fetching page 7 ...
[SUCCESS] Saved 15 fatwas from page 7
[INFO] Fetching page 8 ...
[SUCCESS] Saved 15 fatwas from page 8
[INFO] Fetching page 9 ...
[SUCCESS] Saved 15 fatwas from page 9
[INFO] Fetching page 10 ...
[SUCCESS] Saved 15 fatwas from page 10
[INFO] Fetching page 11 ...
[SUCCESS] Saved 15 fatwas from page 11
[INFO] Fetching page 12 ...
[SUCCESS] Saved 15 fatwas from page 12
[INFO] Fetching page 13 ...
[SUCCESS] Saved 15 fatwas from page 13
[INFO] Fetching page 14 ...
[SUCCESS] Saved 15 fatwas from page 14
[INFO] Fetching page 15 ...
[SUCCESS] Saved 15 f

2-islamQA 

In [ ]:
# ==============================
# Imports
# ==============================
import requests
from bs4 import BeautifulSoup
import time
import csv
from urllib.parse import urljoin


# ==============================
# Scraper Class
# ==============================
class IslamQAScraper:
    def __init__(self, base_url="https://islamqa.info", language="ar", delay=1.5):
        self.base_url = base_url
        self.language = language
        self.delay = delay

        self.session = requests.Session()
        self.session.headers.update({
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
        })

        self.total_requests = 0
        self.articles_scraped = 0

    # --------------------------
    # Fetch Page
    # --------------------------
    def fetch(self, url):
        try:
            response = self.session.get(url, timeout=10)
            response.raise_for_status()
            time.sleep(self.delay)
            self.total_requests += 1
            return response.text
        except Exception as e:
            print(f"❌ Error fetching {url}: {e}")
            return None

    # --------------------------
    # Get Main Categories
    # --------------------------
    def get_categories(self):
        url = f"{self.base_url}/{self.language}/categories/topics"
        html = self.fetch(url)
        if not html:
            return []

        soup = BeautifulSoup(html, "html.parser")
        categories = []

        for item in soup.select("div[data-sut='SUT_category_in_categories_list']"):
            name = item.select_one("div[data-sut='SUT_category_name']")
            link = item.select_one("a[data-sut='SUT_display_category_answers']")

            if not name or not link:
                continue

            categories.append({
                "name": name.get_text(strip=True),
                "url": urljoin(self.base_url, link["href"])
            })

        print(f"📂 Categories found: {len(categories)}")
        return categories

    # --------------------------
    # Get Article Links
    # --------------------------
    def get_articles(self, category_url, max_pages=3):
        article_links = []

        for page in range(1, max_pages + 1):
            url = category_url if page == 1 else f"{category_url}?page={page}"
            print(f"📄 Fetching page {page}")

            html = self.fetch(url)
            if not html:
                break

            soup = BeautifulSoup(html, "html.parser")
            items = soup.select("li[data-gtm='link-list-item-container'] a")

            if not items:
                break

            for a in items:
                article_links.append(urljoin(self.base_url, a["href"]))

        print(f"📄 Total articles collected: {len(article_links)}")
        return article_links

    # --------------------------
    # Extract question & answer
    # --------------------------
    def extract_qa(self, article_url):
        html = self.fetch(article_url)
        if not html:
            return None

        soup = BeautifulSoup(html, "html.parser")

        question = "question not found"
        answer = "answer not found"

        # question
        q_section = soup.find("section", class_="tw-bg-paperquestion")
        if q_section:
            question = q_section.get_text(strip=True)

        # answer
        a_div = soup.find("div", class_="post-body_postBody__TVZCQ")
        if a_div:
            answer = a_div.get_text(strip=True)

        self.articles_scraped += 1

        return {
            "question": question,
            "answer": answer,
            "link": article_url
        }

    # --------------------------
    # Full Pipeline → CSV
    # --------------------------
    def scrape_to_csv(self, csv_filename="islamqa_qa.csv", max_pages=2):
        categories = self.get_categories()
        if not categories:
            print("❌ No categories found")
            return

        with open(csv_filename, "w", newline="", encoding="utf-8-sig") as f:
            writer = csv.DictWriter(
                f, fieldnames=["question", "answer", "link"]
            )
            writer.writeheader()

            for idx, cat in enumerate(categories, 1):
                print(f"\n📂 [{idx}/{len(categories)}] Category: {cat['name']}")
                articles = self.get_articles(cat["url"], max_pages=max_pages)

                for link in articles:
                    qa = self.extract_qa(link)
                    if qa:
                        writer.writerow(qa)
                        print(f"✅ Saved: {qa['question'][:60]}...")

        print("\n" + "=" * 60)
        print("🎉 Scraping Finished")
        print(f"📊 Articles scraped: {self.articles_scraped}")
        print(f"🌐 HTTP requests: {self.total_requests}")
        print(f"💾 File saved: {csv_filename}")
        print("=" * 60)

scraper = IslamQAScraper(language="ar")
scraper.scrape_to_csv(
    csv_filename="islamqa_questions_answers.csv",
    max_pages=1000000000  # زوّدها بعد ما تتأكد إن الدنيا شغالة
)


📂 Categories found: 10

📂 [1/10] Category: العقيدة
📄 Fetching page 1
📄 Fetching page 2
📄 Fetching page 3
📄 Fetching page 4
📄 Fetching page 5
📄 Fetching page 6
📄 Fetching page 7
📄 Fetching page 8
📄 Fetching page 9
📄 Fetching page 10
📄 Fetching page 11
📄 Fetching page 12
📄 Fetching page 13
📄 Fetching page 14
📄 Fetching page 15
📄 Fetching page 16
📄 Fetching page 17
📄 Fetching page 18
📄 Fetching page 19
📄 Fetching page 20
📄 Fetching page 21
📄 Fetching page 22
📄 Fetching page 23
📄 Fetching page 24
📄 Fetching page 25
📄 Fetching page 26
📄 Fetching page 27
📄 Fetching page 28
📄 Fetching page 29
📄 Fetching page 30
📄 Fetching page 31
📄 Fetching page 32
📄 Fetching page 33
📄 Fetching page 34
❌ Error fetching https://islamqa.info/ar/categories/topics/3?page=34: 404 Client Error: Not Found for url: https://islamqa.info/ar/categories/topics/3?page=34
📄 Total articles collected: 327
✅ Saved: السؤال:547394هناك حديث للرسول (لا يسمع صوت المؤذن جن ولا إنس...
✅ Saved: السؤال:500078لماذا يهتم الله بالأمور ال

In [ ]:
import pandas as pd
df=pd.read_csv("islamqa_questions_answers.csv")

In [ ]:
import re
df['question'] = df['question'].apply(lambda x: re.sub(r'^السؤال:\d+', '', x))
df.drop_duplicates(inplace=True)
df.info()
df.head()


<class 'pandas.core.frame.DataFrame'>
Index: 2094 entries, 0 to 2129
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   question  2094 non-null   object
 1   answer    2094 non-null   object
 2   link      2094 non-null   object
dtypes: object(3)
memory usage: 65.4+ KB


,question,answer,link
0,هناك حديث للرسول (لا يسمع صوت المؤذن جن ولا إن...,أولًا:يستعمل (السمع) في لغة العرب، وفي القرآن ...,https://islamqa.info/ar/answers/547394
1,لماذا يهتم الله بالأمور الصغيرة، مثل: الحجاب؟ ...,أولًا:يحق لك أيتها السائلة الكريمة أن تفخري أع...,https://islamqa.info/ar/answers/500078
2,هل يجوز كتابة الأمنيات لما يكتمل القمر ؟ وكيف ...,كتابة الأمنيات عند اكتمال القمر: شعبة من التنج...,https://islamqa.info/ar/answers/438733
3,"كنت أتابع محاضرة يفترض أنها لأخذ العلم , ورد ف...",أولا:الله تعالى هو الخالق، وما سواه مخلوق، ولا...,https://islamqa.info/ar/answers/259874
4,قال تعالى: (أَلَهُمْ أَرْجُلٌۭ يَمْشُونَ بِهَا...,أولًا:يقول الله تعالى في سورة الأعراف:إِنَّ ال...,https://islamqa.info/ar/answers/585602


In [ ]:
df.to_csv("islamqa_questions_answers_cleaned.csv", index=False, encoding="utf-8-sig")

3-islam web

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import time

headers = {
    "User-Agent": "Mozilla/5.0"
}

csv_file = open("islam_web_fatwas_2.csv", "a", newline="", encoding="utf-8-sig")
writer = csv.writer(csv_file)

# write header once
if csv_file.tell() == 0:
    writer.writerow(["question", "answer", "URL"])

for fatwa_id in range(59432, 85001):   # 👈 هنا الـ for loop اللي بتدور عليه
    url = f"https://www.islamweb.net/ar/fatwa/{fatwa_id}/"

    try:
        r = requests.get(url, headers=headers, timeout=10)
        r.encoding = "utf-8"
        if r.status_code != 200:
            continue

        soup = BeautifulSoup(r.text, "html.parser")

        # question
        q_block = soup.find("div", class_="mainitem quest-fatwa")
        if not q_block:
            continue

        question = q_block.find("div", itemprop="text")
        question_text = question.get_text(" ", strip=True) if question else ""

        # answer
        a_block = soup.find("div", itemprop="acceptedanswer")
        if not a_block:
            continue

        answer = a_block.find("div", itemprop="text")
        answer_text = answer.get_text(" ", strip=True) if answer else ""

        if question_text and answer_text:
            writer.writerow([question_text, answer_text, url])
            csv_file.flush()  # 🔥 يحفظ مباشرة

        print(f"✅ scraped {fatwa_id}")

        time.sleep(1.5)  # احترام الموقع

    except Exception as e:
        print(f"❌ {fatwa_id} | {e}")
        continue

csv_file.close()


✅ scraped 59432
✅ scraped 59433
✅ scraped 59434
✅ scraped 59435
✅ scraped 59436
✅ scraped 59438
✅ scraped 59439
✅ scraped 59440
✅ scraped 59441
✅ scraped 59444
✅ scraped 59446
✅ scraped 59448
✅ scraped 59449
✅ scraped 59450
✅ scraped 59451
✅ scraped 59453
✅ scraped 59457
✅ scraped 59458
✅ scraped 59459
✅ scraped 59460
✅ scraped 59462
✅ scraped 59463
✅ scraped 59464
✅ scraped 59466
✅ scraped 59467
✅ scraped 59470
✅ scraped 59471
✅ scraped 59472
✅ scraped 59473
✅ scraped 59475
✅ scraped 59476
✅ scraped 59477
✅ scraped 59478
✅ scraped 59479
✅ scraped 59480
✅ scraped 59482
✅ scraped 59484
✅ scraped 59488
✅ scraped 59489
✅ scraped 59491
✅ scraped 59495
✅ scraped 59496
✅ scraped 59500
✅ scraped 59501
✅ scraped 59502
✅ scraped 59504
✅ scraped 59505
✅ scraped 59506
✅ scraped 59507
✅ scraped 59509
✅ scraped 59510
✅ scraped 59514
✅ scraped 59515
✅ scraped 59516
✅ scraped 59517
✅ scraped 59519
✅ scraped 59522
✅ scraped 59523
✅ scraped 59524
✅ scraped 59525
✅ scraped 59526
✅ scraped 59527
✅ scrape

KeyboardInterrupt: 

### Importing Data

In [ ]:
binbaz.columns = ['question', 'answer', 'URL']
binbaz.head()
binbaz.to_csv("fatwas_binbaz.csv", index=False, encoding="utf-8-sig")

islamqa.columns = ['question', 'answer', 'URL']
islamqa.head()
islamqa.to_csv("islamqa_questions_answers_cleaned.csv", index=False, encoding="utf-8-sig")

In [ ]:
import pandas as pd

binbaz = pd.read_csv("fatwas_binbaz.csv")
islamqa=pd.read_csv("islamqa_questions_answers_cleaned.csv")
islamweb=pd.read_csv("islam_web_fatwas.csv")
islamweb_2=pd.read_csv("islam_web_fatwas_2.csv")

islamweb=pd.concat([islamweb, islamweb_2], ignore_index=True)
data = pd.concat([binbaz, islamqa, islamweb], ignore_index=True)
data.tail()

,question,answer,URL
51468,أنا عند الضحك أتبول على نفسي تبولا كثيرا فهل أ...,الحمد لله، والصلاة والسلام على رسول الله، وعلى...,https://www.islamweb.net/ar/fatwa/73923/
51469,سؤالي ليس استشارة وإنما معلومة وددت معرفتها وأ...,الحمد لله، والصلاة والسلام على رسول الله، وعلى...,https://www.islamweb.net/ar/fatwa/73925/
51470,هل الكرامات تقع لأشخاص صالحين بشكل مطلق بحيث ل...,الحمد لله، والصلاة والسلام على رسول الله، وعلى...,https://www.islamweb.net/ar/fatwa/73927/
51471,كيفية التعامل مع جيران لايملكون من الأخلاق ذرة...,الحمد لله، والصلاة والسلام على رسول الله، وعلى...,https://www.islamweb.net/ar/fatwa/73929/
51472,سؤالي عبارة عن عدة أجزاء أبدؤها بوصف نفسي أنا ...,الحمد لله، والصلاة والسلام على رسول الله، وعلى...,https://www.islamweb.net/ar/fatwa/73932/


In [4]:
data=data.dropna()
data=data.drop_duplicates()
data.info()
data.to_csv("colleted.csv", index=False)


<class 'pandas.core.frame.DataFrame'>
Index: 51470 entries, 0 to 51472
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   question  51470 non-null  object
 1   answer    51470 non-null  object
 2   URL       51470 non-null  object
dtypes: object(3)
memory usage: 1.6+ MB



### Text preprocessing

In [ ]:
import re

def normalize_arabic(text):
    # إزالة التشكيل (diacritics)
    text = re.sub(r'[\u0617-\u061A\u064B-\u0652]', '', text)
    
    # إزالة التمديد (ـ)
    text = re.sub(r'[\u0640]', '', text)
    
    # توحيد الهمزات
    text = re.sub(r'[إأآا]', 'ا', text)
    
    # توحيد تاء مربوطة وتاء هاء (ة → ه)
    text = re.sub(r'ة', 'ه', text)
    
    # توحيد ياء وألف مقصورة (ى → ي)
    text = re.sub(r'ى', 'ي', text)
    
    # إزالة علامات الترقيم باستثناء علامات الاستفهام والنقطة
    text = re.sub(r'[!"#$%&\'()*+,\-/:;<=>?@\[\\\]^_`{|}~]', ' ', text)
    
    # استبدال الفاصلة العربية والإنجليزية بمسافة
    text = re.sub(r'[،,]', ' ', text)
    
    # توحيد المسافات
    text = re.sub(r'\s+', ' ', text).strip()
    
    # إزالة أرقام (لو مش مهمة)
    text = re.sub(r'\d+', '', text)
    
    
    return text

data['question'] = data['question'].apply(normalize_arabic)


### stopwords

In [ ]:
arabic_stopwords = set("""
إذ إذا إذما إذن إلى أنه إنها أن أو أي أى أين 
بعض قبل بل بين بما بها بهاب بهاه بهاك 
بهاه بيد بسبب بشكل كانت كنت كيف كان كان 
كل لذلك لذلكل لديهم لديهمهم لما لن لذا مثل منذ مع معك 
معه معهما معه المزيد مثل ما ماذا متى ممن من هل هي هو 
و و لكن ولدى لو لما نعم نعم نحن هذه هذا هو هي هنا هنالك هناك 
هكذا هذا هؤلاء حيث بحيث سواء سوى كذلك كلا كلما كم كمثل كما
 رب سبحانه وتعالى في على و من ما ما هو الذي التي الذين التي
""".split())

def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word not in arabic_stopwords]
    return ' '.join(filtered_words)

# Example usage
data['question'] = data['question'].apply(remove_stopwords)


### Data embedding


In [ ]:
from sentence_transformers import SentenceTransformer

questions = data['question'].tolist()

# نحمّل الموديل العربي
model = SentenceTransformer("CAMeL-Lab/bert-base-arabic-camelbert-msa")

# نعمل embeddings لكل الأسئلة
embeddings = model.encode(questions, convert_to_tensor=True)


c:\Users\hp\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No sentence-transformers model found with name CAMeL-Lab/bert-base-arabic-camelbert-msa. Creating a new one with mean pooling.


In [33]:
import torch
torch.save(embeddings, "embeddings.pt")


In [41]:
import torch
embeddings = torch.load("embeddings.pt")

# التأكد من نوع البيانات
print(type(embeddings))
print(embeddings.shape)

<class 'torch.Tensor'>
torch.Size([4735, 768])


### Cosin Similarity

In [ ]:
from sentence_transformers import SentenceTransformer

questions = data['question'].tolist()

# نحمّل الموديل العربي
model = SentenceTransformer("CAMeL-Lab/bert-base-arabic-camelbert-msa")





import torch

def search(query, top_k=5):
    # نعمل embedding للسؤال اللي هتسأله
    query_emb = model.encode(query, convert_to_tensor=True)

    # نحسب التشابه
    cos_sim = torch.nn.functional.cosine_similarity(query_emb, embeddings)

    # نجيب أعلى top_k
    top_k_idx = torch.topk(cos_sim, k=top_k).indices

    return data.iloc[top_k_idx][["question","answer","URL"]]

query = "حكم التمائم"
results = search(query)
print(results)


No sentence-transformers model found with name CAMeL-Lab/bert-base-arabic-camelbert-msa. Creating a new one with mean pooling.


                                question  \
3964             حكم الدعاء علي غير وضوء   
179              حكم مس المصحف بغير وضوء   
863               حكم الوضوء داخل الحمام   
4377             حكم لمس امراه بعد وضوئه   
24391  حكم الاستماع للقران علي غير وضوء؟   

                                                  answer  \
3964   حكم الدعاء على غير وضوء\nالسؤال:\nفي بعض الليا...   
179    حكم مس المصحف بغير وضوء\nس: ما حكم مس المصحف ب...   
863    حكم الوضوء داخل الحمام\nس:\nما حكم من يتوضأ دا...   
4377   حكم من لمس امرأة بعد وضوئه\nالسؤال:\nهل لمس ال...   
24391  الحمد لله، والصلاة والسلام على رسول الله، وعلى...   

                                                     URL  
3964   https://binbaz.org.sa/fatwas/19923/%D8%AD%D9%8...  
179    https://binbaz.org.sa/fatwas/928/%D8%AD%D9%83%...  
863    https://binbaz.org.sa/fatwas/3588/%D8%AD%D9%83...  
4377   https://binbaz.org.sa/fatwas/20344/%D8%AD%D9%8...  
24391           https://www.islamweb.net/ar/fatwa/32424/  


### Evaluation

In [ ]:
import time
import torch

def search_indices(query, top_k=5):
    query_emb = model.encode(query, convert_to_tensor=True)
    cos_sim = torch.nn.functional.cosine_similarity(query_emb, embeddings)
    top_k_idx = torch.topk(cos_sim, k=top_k).indices
    return top_k_idx.cpu().tolist(), cos_sim

def evaluate(queries, true_indices, top_k=5):
    total_queries = len(queries)
    top1_correct = 0
    top5_correct = 0
    reciprocal_ranks = []
    cosine_sims = []
    inference_times = []
    
    for i, query in enumerate(queries):
        start_time = time.time()
        top_k_idx, cos_sim = search_indices(query, top_k=top_k)
        end_time = time.time()
        
        inference_times.append(end_time - start_time)
        
        true_idx = true_indices[i]
        
        # Average cosine similarity with top_k results
        cosine_sims.append(cos_sim[top_k_idx].mean().item())
        
        # Top-1 accuracy
        if top_k_idx[0] == true_idx:
            top1_correct += 1
        
        # Top-5 accuracy
        if true_idx in top_k_idx:
            top5_correct += 1
        
        # Reciprocal Rank
        if true_idx in top_k_idx:
            rank = top_k_idx.index(true_idx) + 1
            reciprocal_ranks.append(1.0 / rank)
        else:
            reciprocal_ranks.append(0.0)
    
    top1_accuracy = top1_correct / total_queries
    top5_accuracy = top5_correct / total_queries
    mrr = sum(reciprocal_ranks) / total_queries
    avg_cosine_sim = sum(cosine_sims) / total_queries
    avg_inference_time = sum(inference_times) / total_queries
    
    return {
        'Top-1 Accuracy': top1_accuracy,
        'Top-5 Accuracy': top5_accuracy,
        'MRR': mrr,
        'Average Cosine Similarity': avg_cosine_sim,
        'Average Inference Time (s)': avg_inference_time
    }

# مثال تشغيل التقييم
test_queries = data['question'].tolist()[:1000]  # اختبر أول 100 سؤال أو غير العدد براحتك
true_indices = list(range(len(test_queries)))    # لو الإجابة الصحيحة هي نفسها السؤال عند نفس الإندكس

results = evaluate(test_queries, true_indices)
print(results)


{'Top-1 Accuracy': 0.988, 'Top-5 Accuracy': 1.0, 'MRR': 0.994, 'Average Cosine Similarity': 0.8437313198447227, 'Average Inference Time (s)': 0.24988473892211915}


In [ ]:
import subprocess

# Path للـ Ollama local
OLLAMA_PATH = "ollama"  # لو في PATH، أو استخدم المسار الكامل

MODEL = "gemma3:1b"  # أو "gemma3:1b" لو جهازك أضعف

def generate_response(fatwa_text, question):
    prompt = f"""
انت شيخ مفتي. اقرأ الفتوى التالية ولاحظ محتواها. استخدمها فقط للإجابة على السؤال التالي بإيجاز ووضوح:

الفتوى:
{fatwa_text}

السؤال:
{question}

الإجابة (مباشرة وواضحة بدون تكرار نص الفتوى):
"""
    result = subprocess.run(
        [OLLAMA_PATH, "run", MODEL],
        input=prompt,
        text=True,
        capture_output=True,
        encoding='utf-8'  # ده مهم جدًا
    )


    return result.stdout.strip()



In [ ]:
question = "حكم التمائم"
fatwa_text = search(question).iloc[1]['answer']
response = generate_response(fatwa_text, question)

print("\nResponse:\n", response)

STDOUT: 'لا يجوز الصلاة بدون وضوء.\n\n\n'
STDERR: '\x1b[?2026h\x1b[?25l\x1b[1G⠋ \x1b[K\x1b[?25h\x1b[?2026l\x1b[?2026h\x1b[?25l\x1b[1G⠹ \x1b[K\x1b[?25h\x1b[?2026l\x1b[?2026h\x1b[?25l\x1b[1G⠸ \x1b[K\x1b[?25h\x1b[?2026l\x1b[?2026h\x1b[?25l\x1b[1G⠼ \x1b[K\x1b[?25h\x1b[?2026l\x1b[?2026h\x1b[?25l\x1b[1G⠼ \x1b[K\x1b[?25h\x1b[?2026l\x1b[?2026h\x1b[?25l\x1b[1G⠦ \x1b[K\x1b[?25h\x1b[?2026l\x1b[?2026h\x1b[?25l\x1b[1G⠦ \x1b[K\x1b[?25h\x1b[?2026l\x1b[?2026h\x1b[?25l\x1b[1G⠇ \x1b[K\x1b[?25h\x1b[?2026l\x1b[?2026h\x1b[?25l\x1b[1G⠏ \x1b[K\x1b[?25h\x1b[?2026l\x1b[?2026h\x1b[?25l\x1b[1G⠋ \x1b[K\x1b[?25h\x1b[?2026l\x1b[?2026h\x1b[?25l\x1b[1G⠙ \x1b[K\x1b[?25h\x1b[?2026l\x1b[?2026h\x1b[?25l\x1b[1G⠹ \x1b[K\x1b[?25h\x1b[?2026l\x1b[?2026h\x1b[?25l\x1b[1G⠸ \x1b[K\x1b[?25h\x1b[?2026l\x1b[?2026h\x1b[?25l\x1b[1G⠼ \x1b[K\x1b[?25h\x1b[?2026l\x1b[?2026h\x1b[?25l\x1b[1G⠼ \x1b[K\x1b[?25h\x1b[?2026l\x1b[?2026h\x1b[?25l\x1b[1G⠴ \x1b[K\x1b[?25h\x1b[?2026l\x1b[?2026h\x1b[?25l\x1b[1G⠧ \x1b[K\x1b[?25h\x1b[?2026l\x1b[?2026h\x1